# Accessing info with `qiskit-superstaq`
Below we'll go over what information on your account, you jobs, and backends to which you have access that you can retrieve from Superstaq.

## Imports and API Token

As usual, we'll begin with importing requirements and setting up access to Superstaq. This tutorial uses `qiskit-superstaq`, our Superstaq client for Qiskit. You can install it and relevant dependencies by running `pip install qiskit-superstaq[examples]`.

In [ ]:
%pip install qiskit-superstaq[examples]

In [4]:
# requirements to use qiskit-superstaq
import qiskit
import qiskit_superstaq as qss

Now, we instantiate a provider in `qiskit-superstaq` with `SuperstaQProvider()`. Supply the Superstaq API token by providing the token as an argument of `qss.SuperstaQProvider()` or setting it as an environment variable (see [this guide](/get_started/basics/basics_qss.html)).

In [2]:
# for qiskit-superstaq: provider for SuperstaQ backend
provider = qss.SuperstaQProvider()

## Account Information

The `provider` class gives you a means to retrieve SuperstaQ account information. Regarding your account, you can use `provider` to retrieve your Superstaq balance.

In [3]:
provider.get_balance()

'$200.00'

If are interested in increasing your balance or have more information on your user role, please reach out to us at superstaq@infleqtion.com or join our [Slack workspace](https://join.slack.com/t/superstaq/shared_invite/zt-1wr6eok5j-fMwB7dPEWGG~5S474xGhxw).

## Backend Information
In addition to account information, the`provider` object also gives you a list of all the devices and simulators to which you have access, as well as additional information about those backends.
* `backends()`: Retrieves a list of available backends
    - <mark><span style="color:red">filter backends?</span></mark>
* `get_backend("<backend_name>")`: Select your target backend, where `<backend_name>` is the name of the desired device
* 

In [27]:
backend = provider.get_backend("ibmq_lagos_qpu")
backend.target_info()

{'num_qubits': 7,
 'target': 'ibmq_lagos_qpu',
 'native_gate_set': ['id', 'rz', 'sx', 'x', 'cx', 'reset'],
 'max_experiments': 300,
 'coupling_map': [[0, 1],
  [1, 0],
  [1, 2],
  [1, 3],
  [2, 1],
  [3, 1],
  [3, 5],
  [4, 5],
  [5, 3],
  [5, 4],
  [5, 6],
  [6, 5]],
 'max_shots': 32000,
 'processor_type': {'family': 'Falcon', 'revision': '5.11', 'segment': 'H'},
 'open_pulse': True}

In [28]:
backend.status().to_dict()

{'backend_name': 'ibmq_lagos_qpu',
 'backend_version': '1',
 'operational': True,
 'pending_jobs': 0,
 'status_msg': ''}

In [9]:
provider.backends() # waiting on PR from Paige to get the version where compile-only and run are separated

[<SuperstaQBackend('aws_dm1_simulator')>,
 <SuperstaQBackend('aws_sv1_simulator')>,
 <SuperstaQBackend('aws_tn1_simulator')>,
 <SuperstaQBackend('cq_hilbert_qpu')>,
 <SuperstaQBackend('cq_hilbert_simulator')>,
 <SuperstaQBackend('hqs_lt-s1-apival_qpu')>,
 <SuperstaQBackend('hqs_lt-s1_qpu')>,
 <SuperstaQBackend('hqs_lt-s1_simulator')>,
 <SuperstaQBackend('hqs_lt-s2_qpu')>,
 <SuperstaQBackend('ibmq_belem_qpu')>,
 <SuperstaQBackend('ibmq_extended-stabilizer_simulator')>,
 <SuperstaQBackend('ibmq_guadalupe_qpu')>,
 <SuperstaQBackend('ibmq_jakarta_qpu')>,
 <SuperstaQBackend('ibmq_lagos_qpu')>,
 <SuperstaQBackend('ibmq_lima_qpu')>,
 <SuperstaQBackend('ibmq_manila_qpu')>,
 <SuperstaQBackend('ibmq_mps_simulator')>,
 <SuperstaQBackend('ibmq_nairobi_qpu')>,
 <SuperstaQBackend('ibmq_perth_qpu')>,
 <SuperstaQBackend('ibmq_qasm_simulator')>,
 <SuperstaQBackend('ibmq_quito_qpu')>,
 <SuperstaQBackend('ibmq_stabilizer_simulator')>,
 <SuperstaQBackend('ibmq_statevector_simulator')>,
 <SuperstaQBackend(

## Job Information
Jobs submitted through SuperstaQ contain the following information:
* `job_id()`: Unique identifier for the job
* `status()`: Status of the job (either Queued, Running, Done)
* `backend()`: Device the job was run on
* `result().get_counts()`: Counts from the result of the job run
* circuit? tag for a job, retrieve jobs by user/tag

In [3]:
# using Qiskit
qc = qiskit.QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

backend = provider.get_backend("ibmq_qasm_simulator")
job_qss = backend.run(qc, shots=100)
results_qss = job_qss.result().get_counts()


In [7]:
print(job_qss.result())

Result(backend_name='ibmq_qasm_simulator', backend_version='n/a', qobj_id='-1', job_id='daec9b20-f097-443f-9b0e-103a0aa8fd9e', success=True, results=[ExperimentResult(shots=100, success=True, meas_level=2, data=ExperimentResultData(counts={'00': 53, '11': 47}))], date=None, status=None, header=None)
